In [8]:
import numpy as np
import pandas as pd
from math import factorial
from itertools import combinations_with_replacement

# Set pandas display options to ensure all columns are visible
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)       # Increase display width

def generate_macrostates(num_particles, total_energy):
    """
    Generate all possible macrostates for a given number of particles and total energy.
    A macrostate is represented as a tuple (n0, n1, n2, ..., nk), where ni is the number
    of particles in the i-th energy level, and the sum of ni equals num_particles.
    """
    # Generate all possible combinations of energy levels that sum to total_energy
    energy_levels = list(combinations_with_replacement(range(total_energy + 1), num_particles))
    
    # Filter combinations where the sum of energies equals total_energy
    macrostates = []
    for state in energy_levels:
        if sum(state) == total_energy:
            # Count the number of particles in each energy level
            energy_counts = {}
            for energy in state:
                energy_counts[energy] = energy_counts.get(energy, 0) + 1
            macrostates.append(energy_counts)
    
    return macrostates

def calculate_microstates(macrostate, num_particles):
    """
    Calculate the number of microstates for a given macrostate.
    """
    # Extract the counts of particles in each energy level
    counts = list(macrostate.values())
    
    # Calculate the number of microstates using the formula N! / (n0! n1! n2! ... nk!)
    numerator = factorial(num_particles)
    denominator = np.prod([factorial(count) for count in counts])
    return numerator // denominator

def generate_table(num_particles, total_energy):
    """
    Generate a table similar to Table BD-1 for the given number of particles and total energy.
    Add a sum row and the n(E_i) row at the bottom of the table.
    """
    # Generate all macrostates
    macrostates = generate_macrostates(num_particles, total_energy)
    
    # Initialize a DataFrame to store the results
    columns = [f"{i}ΔE" for i in range(total_energy + 1)] + ["Number of Microstates"]
    df = pd.DataFrame(columns=columns)
    
    # Populate the DataFrame
    for i, macrostate in enumerate(macrostates):
        row = [macrostate.get(energy, 0) for energy in range(total_energy + 1)]
        row.append(calculate_microstates(macrostate, num_particles))
        df.loc[i + 1] = row  # Macrostate numbering starts at 1
    
    # Calculate the total number of microstates
    total_microstates = df["Number of Microstates"].sum()
    
    # Calculate the n(E_i) row
    n_Ei = []
    for energy in range(total_energy + 1):
        weighted_sum = (df[f"{energy}ΔE"] * df["Number of Microstates"]).sum() / total_microstates
        n_Ei.append(round(weighted_sum, 2))  # Round to 2 decimal places for readability


    # Add a sum row at the bottom
    sum_row = df.iloc[:-1].sum(axis=0)  # Sum each column (excluding the n(E_i) row)
    sum_row.name = "Sum"                # Name the sum row
    df = pd.concat([df, sum_row.to_frame().T])  # Append the sum row to the DataFrame

    # Add the n(E_i) row to the DataFrame
    n_Ei_row = pd.Series(n_Ei + [""], index=df.columns, name="n(E_i)")
    df = pd.concat([df, n_Ei_row.to_frame().T])
    

    
    return df

# Example usage
num_particles = 6
total_energy = 8
table = generate_table(num_particles, total_energy)
print(table)

         0ΔE   1ΔE   2ΔE   3ΔE   4ΔE   5ΔE   6ΔE   7ΔE  8ΔE Number of Microstates
1          5     0     0     0     0     0     0     0    1                     6
2          4     1     0     0     0     0     0     1    0                    30
3          4     0     1     0     0     0     1     0    0                    30
4          4     0     0     1     0     1     0     0    0                    30
5          4     0     0     0     2     0     0     0    0                    15
6          3     2     0     0     0     0     1     0    0                    60
7          3     1     1     0     0     1     0     0    0                   120
8          3     1     0     1     1     0     0     0    0                   120
9          3     0     2     0     1     0     0     0    0                    60
10         3     0     1     2     0     0     0     0    0                    60
11         2     3     0     0     0     1     0     0    0                    60
12         2    